medallion_staging--> bronze table


In [0]:
spark.read.csv(path="/Volumes/dev_catalog/default/retail_staging/customers/customers.csv",
               header=True,
               inferSchema=True).write.format("delta").mode("overwrite").saveAsTable("dev_catalog.bronze.customers")

In [0]:
spark.read.table("dev_catalog.bronze.customers").display()

In [0]:
spark.read.csv(path="/Volumes/dev_catalog/default/retail_staging/orders/",
               header=True,
               inferSchema=True).write.format("delta").mode("overwrite").saveAsTable("dev_catalog.bronze.orders")

In [0]:
spark.read.table("dev_catalog.bronze.orders").display()

Silver Layer

Customers

In [0]:
from pyspark.sql.functions import col
spark.read.table("dev_catalog.bronze.customers").filter(col("CustomerName").isNotNull()).write.mode("append").saveAsTable("dev_catalog.silver.customers")
spark.read.table("dev_catalog.silver.customers").display()


orders

In [0]:
from pyspark.sql.functions import col
spark.read.table("dev_catalog.bronze.orders").filter(col("CustomerID").isNotNull()).write.mode("append").saveAsTable("dev_catalog.silver.orders")
spark.read.table("dev_catalog.silver.orders").display()


Gold Layer

In [0]:
customers_df= spark.read.table("dev_catalog.silver.customers")
orders_df= spark.read.table("dev_catalog.silver.orders")
joined_df= customers_df.join(orders_df,customers_df.CustomerID==orders_df.CustomerID,"inner")
# joined_df.select(customers_df.CustomerID,"OrderID").groupBy("CustomerID").display()
# joined_df.write.mode("overwrite").saveAsTable("dev_catalog.silver.customer_orders")

joined_df.select(customers_df.CustomerID,"OrderID").groupBy("CustomerID").count().write.mode("append").saveAsTable("dev_catalog.gold.sales_agg")
spark.read.table("dev_catalog.gold.sales_agg").display()